<a href="https://colab.research.google.com/github/Suraba03/ml_san_francisco_permits/blob/main/san_fancisco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# import data and first glance

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/san_francisco/Building_Permits.csv')

<ipython-input-12-5a9255161ba3>:1: DtypeWarning: Columns (22,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/san_francisco/Building_Permits.csv')


In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df.head(2)

,PermitNumber,PermitType,PermitTypeDefinition,PermitCreationDate,Block,Lot,StreetNumber,StreetNumberSuffix,StreetName,StreetSuffix,Unit,UnitSuffix,Description,CurrentStatus,CurrentStatusDate,FiledDate,IssuedDate,CompletedDate,FirstConstructionDocumentDate,StructuralNotification,NumberofExistingStories,NumberofProposedStories,VoluntarySoft-StoryRetrofit,FireOnlyPermit,PermitExpirationDate,EstimatedCost,RevisedCost,ExistingUse,ExistingUnits,ProposedUse,ProposedUnits,Plansets,TIDFCompliance,ExistingConstructionType,ExistingConstructionTypeDescription,ProposedConstructionType,ProposedConstructionTypeDescription,SitePermit,SupervisorDistrict,Neighborhoods-AnalysisBoundaries,Zipcode,Location,RecordID
0,201505065519,4,sign - erect,05/06/2015,0326,023,140,NaN,Ellis,St,NaN,NaN,"ground fl facade: to erect illuminated, electr...",expired,12/21/2017,05/06/2015,11/09/2015,NaN,11/09/2015,NaN,6.0,NaN,NaN,NaN,11/03/2016,4000.0,4000.0,tourist hotel/motel,143.0,NaN,NaN,2.0,NaN,3.0,constr type 3,NaN,NaN,NaN,3.0,Tenderloin,94102.0,"(37.785719256680785, -122.40852313194863)",1380611233945
1,201604195146,4,sign - erect,04/19/2016,0306,007,440,NaN,Geary,St,0.0,NaN,remove (e) awning and associated signs.,issued,08/03/2017,04/19/2016,08/03/2017,NaN,08/03/2017,NaN,7.0,NaN,NaN,NaN,12/03/2017,1.0,500.0,tourist hotel/motel,NaN,NaN,NaN,2.0,NaN,3.0,constr type 3,NaN,NaN,NaN,3.0,Tenderloin,94102.0,"(37.78733980600732, -122.41063199757738)",1420164406718


In [ ]:
df.shape

(198900, 43)

In [46]:
df.PermitType.unique()

array([4, 3, 8, 6, 7, 1, 5, 2])

In [47]:
df.PermitTypeDefinition.unique()

array(['sign - erect', 'additions alterations or repairs',
       'otc alterations permit', 'demolitions', 'wall or painted sign',
       'new construction', 'grade or quarry or fill or excavate',
       'new construction wood frame'], dtype=object)

In [51]:
df[["PermitType", "PermitTypeDefinition"]].drop_duplicates(keep='first')

,PermitType,PermitTypeDefinition
0,4,sign - erect
2,3,additions alterations or repairs
3,8,otc alterations permit
4,6,demolitions
55,7,wall or painted sign
83,1,new construction
152,5,grade or quarry or fill or excavate
164,2,new construction wood frame


# preprocessing

let's remove white spaces from columns names

In [ ]:
df = df.rename(lambda name: name.replace(" ", ""), axis='columns')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198900 entries, 0 to 198899
Data columns (total 43 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   PermitNumber                         198900 non-null  object 
 1   PermitType                           198900 non-null  int64  
 2   PermitTypeDefinition                 198900 non-null  object 
 3   PermitCreationDate                   198900 non-null  object 
 4   Block                                198900 non-null  object 
 5   Lot                                  198900 non-null  object 
 6   StreetNumber                         198900 non-null  int64  
 7   StreetNumberSuffix                   2216 non-null    object 
 8   StreetName                           198900 non-null  object 
 9   StreetSuffix                         196132 non-null  object 
 10  Unit                                 29479 non-null   float64
 11  UnitSuffix   

let's split features list on two: numerical and categorical

In [42]:
num_cols = [col for col in df.columns if "int" in str(df[col].dtype) or "float" in str(df[col].dtype)]
cat_cols = [col for col in df.columns if "object" == str(df[col].dtype)]
print(num_cols)
print(cat_cols)

['PermitType', 'StreetNumber', 'Unit', 'NumberofExistingStories', 'NumberofProposedStories', 'EstimatedCost', 'RevisedCost', 'ExistingUnits', 'ProposedUnits', 'Plansets', 'ExistingConstructionType', 'ProposedConstructionType', 'SupervisorDistrict', 'Zipcode', 'RecordID']
['PermitNumber', 'PermitTypeDefinition', 'PermitCreationDate', 'Block', 'Lot', 'StreetNumberSuffix', 'StreetName', 'StreetSuffix', 'UnitSuffix', 'Description', 'CurrentStatus', 'CurrentStatusDate', 'FiledDate', 'IssuedDate', 'CompletedDate', 'FirstConstructionDocumentDate', 'StructuralNotification', 'VoluntarySoft-StoryRetrofit', 'FireOnlyPermit', 'PermitExpirationDate', 'ExistingUse', 'ProposedUse', 'TIDFCompliance', 'ExistingConstructionTypeDescription', 'ProposedConstructionTypeDescription', 'SitePermit', 'Neighborhoods-AnalysisBoundaries', 'Location']


just a sanity check

In [45]:
set(num_cols+cat_cols) == set(df.columns)

True

## deal with missing values

In [40]:
for i in df.columns:
    print(i)

PermitNumber
PermitType
PermitTypeDefinition
PermitCreationDate
Block
Lot
StreetNumber
StreetNumberSuffix
StreetName
StreetSuffix
Unit
UnitSuffix
Description
CurrentStatus
CurrentStatusDate
FiledDate
IssuedDate
CompletedDate
FirstConstructionDocumentDate
StructuralNotification
NumberofExistingStories
NumberofProposedStories
VoluntarySoft-StoryRetrofit
FireOnlyPermit
PermitExpirationDate
EstimatedCost
RevisedCost
ExistingUse
ExistingUnits
ProposedUse
ProposedUnits
Plansets
TIDFCompliance
ExistingConstructionType
ExistingConstructionTypeDescription
ProposedConstructionType
ProposedConstructionTypeDescription
SitePermit
SupervisorDistrict
Neighborhoods-AnalysisBoundaries
Zipcode
Location
RecordID
